In [11]:
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import cv2
import dlib
import os
from tensorflow import keras
import sklearn

# 1- Importation des données

In [14]:
import os
import pandas as pd

# Chemin du dossier principal contenant les sous-dossiers
dataset_dir = "data/train"

# Liste pour stocker les informations
data = []

# Parcourir chaque sous-dossier
for label in os.listdir(dataset_dir):
    subfolder_path = os.path.join(dataset_dir, label)
    
    # Vérifier si c'est bien un dossier
    if os.path.isdir(subfolder_path):
        # Parcourir les fichiers dans le sous-dossier
        for file_name in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file_name)
            
            # Vérifier si c'est bien une image
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                data.append({"image_path": file_path, "label": label})

# Créer un DataFrame Pandas
df = pd.DataFrame(data)

# Sauvegarder dans un fichier CSV
output_csv = "train_labeled_dataset.csv"
df.to_csv(output_csv, index=False)

print(f"Dataset structuré enregistré dans {output_csv}")
print("Données exemple")
print(df[:10])


Dataset structuré enregistré dans train_labeled_dataset.csv
Données exemple
                               image_path  label
0  data/train\angry\Training_10118481.jpg  angry
1  data/train\angry\Training_10120469.jpg  angry
2  data/train\angry\Training_10131352.jpg  angry
3  data/train\angry\Training_10161559.jpg  angry
4   data/train\angry\Training_1021836.jpg  angry
5  data/train\angry\Training_10269675.jpg  angry
6  data/train\angry\Training_10278738.jpg  angry
7  data/train\angry\Training_10290703.jpg  angry
8  data/train\angry\Training_10295477.jpg  angry
9  data/train\angry\Training_10315441.jpg  angry


In [15]:
import os
import pandas as pd

# Chemin du dossier principal contenant les sous-dossiers
dataset_dir = "data/test"

# Liste pour stocker les informations
data = []

# Parcourir chaque sous-dossier
for label in os.listdir(dataset_dir):
    subfolder_path = os.path.join(dataset_dir, label)
    
    # Vérifier si c'est bien un dossier
    if os.path.isdir(subfolder_path):
        # Parcourir les fichiers dans le sous-dossier
        for file_name in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file_name)
            
            # Vérifier si c'est bien une image
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                data.append({"image_path": file_path, "label": label})

# Créer un DataFrame Pandas
df = pd.DataFrame(data)

# Sauvegarder dans un fichier CSV
output_csv = "test_labeled_dataset.csv"
df.to_csv(output_csv, index=False)

print(f"Dataset structuré enregistré dans {output_csv}")
print("Données exemple")
print(df[:10])


Dataset structuré enregistré dans test_labeled_dataset.csv
Données exemple
                                 image_path  label
0  data/test\angry\PrivateTest_10131363.jpg  angry
1  data/test\angry\PrivateTest_10304478.jpg  angry
2   data/test\angry\PrivateTest_1054527.jpg  angry
3  data/test\angry\PrivateTest_10590091.jpg  angry
4   data/test\angry\PrivateTest_1109992.jpg  angry
5  data/test\angry\PrivateTest_11296953.jpg  angry
6  data/test\angry\PrivateTest_12000629.jpg  angry
7  data/test\angry\PrivateTest_12008383.jpg  angry
8  data/test\angry\PrivateTest_12191716.jpg  angry
9   data/test\angry\PrivateTest_1221822.jpg  angry


#1- Normalisation des images

In [18]:
#!pip uninstall dlib
#!pip install dlib

# 2- Prétraitement des données

In [20]:
import dlib
hog_detector = dlib.get_frontal_face_detector()
cnn_detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')

def face_locations(image, model="hog"):


    detector = hog_detector
    cst = 0
    #elif model == "cnn":
    #    detector = cnn_detector
    #    cst = 10

    matches = detector(image,1)
    rects   = []

    for r in matches:
        if model == "cnn":
            r = r.rect
        x = max(r.left(), 0)
        y = max(r.top(), 0)
        w = min(r.right(), image.shape[1]) - x + cst
        h = min(r.bottom(), image.shape[0]) - y + cst
        rects.append((x,y,w,h))

    return rects

In [21]:
def extract_faces(image, model="hog"):

    gray  = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    rects = face_locations(gray, model)
    faces = []

    for (x,y,w,h) in rects:
        cropped = image[y:y+h, x:x+w, :]
        cropped = cv2.resize(cropped, (128,128))
        faces.append(cropped)

    return faces

In [23]:
def show_grid(faces, figsize=(8,8)):
    face = faces[0]
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(face)
    ax.axis('off')

In [24]:
def list_images(basePath, validExts=(".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"), contains=None):

    imagePaths = []

    # loop over the directory structure
    for (rootDir, dirNames, filenames) in os.walk(basePath):
        # loop over the filenames in the current directory
        for filename in filenames:
            # if the contains string is not none and the filename does not contain
            # the supplied string, then ignore the file
            if contains is not None and filename.find(contains) == -1:
                continue

            # determine the file extension of the current file
            ext = filename[filename.rfind("."):].lower()

            # check to see if the file is an image and should be processed
            if ext.endswith(validExts):
                # construct the path to the image and yield it
                imagePath = os.path.join(rootDir, filename).replace(" ", "\\ ")
                imagePaths.append(imagePath)

    return imagePaths

# 2- Estimation du visage

In [26]:
pose68 = dlib.shape_predictor('models/shape_predictor_68_face_landmarks.dat')
pose05 = dlib.shape_predictor('models/shape_predictor_5_face_landmarks.dat')

In [27]:
def face_landmarks(face, model="large"):

    if model == "large":
        predictor = pose68
    elif model == "small":
        predictor = pose05

    if not isinstance(face, list):
        rect = dlib.rectangle(0,0,face.shape[1],face.shape[0])
        return predictor(face, rect)
    else:
        rect = dlib.rectangle(0,0,face[0].shape[1],face[0].shape[0])
        return [predictor(f,rect) for f in face]

In [28]:
def shape_to_coords(shape):
    return np.float32([[p.x, p.y] for p in shape.parts()])

In [29]:
TEMPLATE = np.float32([
    (0.0792396913815, 0.339223741112), (0.0829219487236, 0.456955367943),
    (0.0967927109165, 0.575648016728), (0.122141515615, 0.691921601066),
    (0.168687863544, 0.800341263616), (0.239789390707, 0.895732504778),
    (0.325662452515, 0.977068762493), (0.422318282013, 1.04329000149),
    (0.531777802068, 1.06080371126), (0.641296298053, 1.03981924107),
    (0.738105872266, 0.972268833998), (0.824444363295, 0.889624082279),
    (0.894792677532, 0.792494155836), (0.939395486253, 0.681546643421),
    (0.96111933829, 0.562238253072), (0.970579841181, 0.441758925744),
    (0.971193274221, 0.322118743967), (0.163846223133, 0.249151738053),
    (0.21780354657, 0.204255863861), (0.291299351124, 0.192367318323),
    (0.367460241458, 0.203582210627), (0.4392945113, 0.233135599851),
    (0.586445962425, 0.228141644834), (0.660152671635, 0.195923841854),
    (0.737466449096, 0.182360984545), (0.813236546239, 0.192828009114),
    (0.8707571886, 0.235293377042), (0.51534533827, 0.31863546193),
    (0.516221448289, 0.396200446263), (0.517118861835, 0.473797687758),
    (0.51816430343, 0.553157797772), (0.433701156035, 0.604054457668),
    (0.475501237769, 0.62076344024), (0.520712933176, 0.634268222208),
    (0.565874114041, 0.618796581487), (0.607054002672, 0.60157671656),
    (0.252418718401, 0.331052263829), (0.298663015648, 0.302646354002),
    (0.355749724218, 0.303020650651), (0.403718978315, 0.33867711083),
    (0.352507175597, 0.349987615384), (0.296791759886, 0.350478978225),
    (0.631326076346, 0.334136672344), (0.679073381078, 0.29645404267),
    (0.73597236153, 0.294721285802), (0.782865376271, 0.321305281656),
    (0.740312274764, 0.341849376713), (0.68499850091, 0.343734332172),
    (0.353167761422, 0.746189164237), (0.414587777921, 0.719053835073),
    (0.477677654595, 0.706835892494), (0.522732900812, 0.717092275768),
    (0.569832064287, 0.705414478982), (0.635195811927, 0.71565572516),
    (0.69951672331, 0.739419187253), (0.639447159575, 0.805236879972),
    (0.576410514055, 0.835436670169), (0.525398405766, 0.841706377792),
    (0.47641545769, 0.837505914975), (0.41379548902, 0.810045601727),
    (0.380084785646, 0.749979603086), (0.477955996282, 0.74513234612),
    (0.523389793327, 0.748924302636), (0.571057789237, 0.74332894691),
    (0.672409137852, 0.744177032192), (0.572539621444, 0.776609286626),
    (0.5240106503, 0.783370783245), (0.477561227414, 0.778476346951)])

TPL_MIN, TPL_MAX = np.min(TEMPLATE, axis=0), np.max(TEMPLATE, axis=0)
MINMAX_TEMPLATE = (TEMPLATE - TPL_MIN) / (TPL_MAX - TPL_MIN)

INNER_EYES_AND_BOTTOM_LIP = np.array([39, 42, 57])
OUTER_EYES_AND_NOSE = np.array([36, 45, 33])

In [30]:
def align_faces(images, landmarks, idx=INNER_EYES_AND_BOTTOM_LIP):
    faces = []
    for (img, marks) in zip(images, landmarks):
        imgDim = img.shape[0]
        coords = shape_to_coords(marks)
        H = cv2.getAffineTransform(coords[idx], imgDim * MINMAX_TEMPLATE[idx])
        warped = cv2.warpAffine(img, H, (imgDim, imgDim))
        faces.append(warped)
    return faces

In [31]:
#image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [32]:
processed_images_dir = "processed_images"
def process_and_save_image(row, model="hog"):
    image_path = row["image_path"]
    label = row["label"]
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Détecter et extraire les visages
    faces = extract_faces(image, model)

    if not faces:
        return None  # Aucun visage détecté

    # Extraire les landmarks et aligner
    landmarks = [face_landmarks(face) for face in faces]
    aligned_faces = align_faces(faces, landmarks)

    # Sauvegarder les images alignées
    saved_paths = []
    for i, face in enumerate(aligned_faces):
        processed_path = os.path.join(
            processed_images_dir, f"{label}_{os.path.basename(image_path)}_{i}.jpg"
        )
        cv2.imwrite(processed_path, cv2.cvtColor(face, cv2.COLOR_RGB2BGR))
        saved_paths.append(processed_path)

    return saved_paths

def process_dataset(csv_path, output_csv, model="hog"):
    df = pd.read_csv(csv_path)
    processed_data = []

    for _, row in df.iterrows():
        processed_paths = process_and_save_image(row, model)
        if processed_paths:
            for path in processed_paths:
                processed_data.append({"image_path": path, "label": row["label"]})

    # Sauvegarder le nouveau DataFrame
    processed_df = pd.DataFrame(processed_data)
    processed_df.to_csv(output_csv, index=False)
    print(f"Processed dataset saved to {output_csv}")
    return processed_df



In [20]:
# Traitez les datasets
train_csv = "train_labeled_dataset.csv"
test_csv = "test_labeled_dataset.csv"
processed_train_csv = "processed_train_labeled_dataset.csv"
processed_test_csv = "processed_test_labeled_dataset.csv"

process_dataset(train_csv, processed_train_csv)
process_dataset(test_csv, processed_test_csv)

Processed dataset saved to processed_train_labeled_dataset.csv
Processed dataset saved to processed_test_labeled_dataset.csv


,image_path,label
0,processed_images\angry_PrivateTest_1054527.jpg...,angry
1,processed_images\angry_PrivateTest_1109992.jpg...,angry
2,processed_images\angry_PrivateTest_11296953.jp...,angry
3,processed_images\angry_PrivateTest_12191716.jp...,angry
4,processed_images\angry_PrivateTest_1221822.jpg...,angry
...,...,...
3965,processed_images\sad_PrivateTest_91626978.jpg_...,sad
3966,processed_images\sad_PrivateTest_92077958.jpg_...,sad
3967,processed_images\sad_PrivateTest_92593957.jpg_...,sad
3968,processed_images\sad_PrivateTest_92794448.jpg_...,sad


# 3 - Encodement du visage

In [33]:
import dlib
cnn_encoder = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')

In [34]:
#Permet d'extraire les coordonnées des marquages du visage
def face_encoder(faces):

    landmarks = face_landmarks(faces)

    if not isinstance(faces, list):
        return np.array(cnn_encoder.compute_face_descriptor(faces,landmarks))
    else:
        return np.array([cnn_encoder.compute_face_descriptor(f,l) for f,l in zip(faces,landmarks)])

__________________________
__________________________
__________________________

In [36]:
#pip install facenet-pytorch fer


In [37]:
#!pip install moviepy==1.0.3

In [38]:
import moviepy.editor


In [39]:
#!pip install tensorflow

In [40]:
import torch
import torchvision
import moviepy
import fer
from facenet_pytorch import MTCNN



print(f"Torch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print("FER and facenet-pytorch modules imported successfully!")

Torch version: 2.2.2+cpu
Torchvision version: 0.17.2+cpu
FER and facenet-pytorch modules imported successfully!


In [41]:
detector = fer.FER(mtcnn=True)

In [42]:
processed_train_csv = "train_labeled_dataset.csv"
df = pd.read_csv(processed_train_csv)

In [49]:
import torch

torch.cuda.is_available()

False

In [46]:
emotion_vectors = []
labels = []

for _, row in df.iterrows():
    image_path = row['image_path']
    label = row['label']
    
    # Charger l'image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Détecter les émotions
    emotion_data = detector.detect_emotions(image)
    
    # Si des émotions sont détectées, stocker le vecteur
    if emotion_data:
        emotions = emotion_data[0]['emotions']  # Premier visage détecté
        emotion_vectors.append(list(emotions.values()))
        labels.append(label)

X = np.array(emotion_vectors)
y = np.array(labels)

# 4 - Préparation du modèle d'apprentissage

In [43]:
from tensorflow.keras import utils, layers, models, optimizers

In [56]:
# Sauvegarde dans un fichier .npy
np.save('emotion_vectors.npy', X)

# Sauvegarde dans un fichier .csv (facile à lire)
np.savetxt('emotion_vectors.csv', X, delimiter=',')

NameError: name 'X' is not defined

In [4]:
# Charger depuis le fichier .npy
X = np.load('emotion_vectors.npy')

# Charger depuis le fichier .csv
X = np.loadtxt('emotion_vectors.csv', delimiter=',')

In [75]:
from sklearn.preprocessing import LabelEncoder

# Encoder les labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Conversion en labels numériques

np.save('label_classes.npy', label_encoder.classes_)

NameError: name 'y' is not defined

In [6]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Load the saved classes
classes = np.load('label_classes.npy')

# Reinitialize the LabelEncoder and set its classes
label_encoder = LabelEncoder()
label_encoder.classes_ = classes

In [79]:
from sklearn.model_selection import train_test_split

# Diviser en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

NameError: name 'y_encoded' is not defined

In [59]:
#Initialisation du model + entrainement KNN
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='euclidean')
# GridSearchCV avec cross-validation
grid_search_knn = GridSearchCV(knn_model, param_grid_knn, cv=5, scoring='accuracy', n_jobs=-1)

# Entraînement
grid_search_knn.fit(X_train, y_train)

prediction_knn = grid_search_knn.predict(X_test)
print(sklearn.metrics.classification_report(y_test, prediction_knn))

              precision    recall  f1-score   support

           0       0.44      0.38      0.41       668
           1       0.65      0.22      0.33        90
           2       0.27      0.20      0.23       655
           3       0.68      0.77      0.72      1234
           4       0.40      0.49      0.44       871
           5       0.34      0.33      0.33       734
           6       0.63      0.60      0.62       579

    accuracy                           0.49      4831
   macro avg       0.49      0.43      0.44      4831
weighted avg       0.48      0.49      0.48      4831



In [61]:
import joblib

# Enregistrer le modèle GridSearchCV
joblib.dump(grid_search_knn, 'grid_search_knn.pkl')
print("Le modèle a été enregistré sous le nom 'grid_search_knn.pkl'")


Le modèle a été enregistré sous le nom 'grid_search_knn.pkl'


In [8]:
from sklearn.metrics import classification_report
import joblib
# Charger le modèle enregistré
grid_search_knn = joblib.load('grid_search_knn.pkl')
print("Le modèle KNN a été chargé avec succès")

# Faire des prédictions sur X_test
loaded_predictions = grid_search_knn.predict(X_test)

# Afficher le rapport de classification
print(classification_report(y_test, loaded_predictions))


Le modèle KNN a été chargé avec succès


NameError: name 'X_test' is not defined

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Paramètres pour le SVC
param_grid_svc = {
    'C': [0.1, 1, 10, 100],  # Pénalité de l'erreur
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Types de noyaux
    'degree': [3, 4, 5],  # Utilisé si kernel=poly
    'gamma': ['scale', 'auto']  # Paramètre de la fonction noyau
}

# Initialisation du modèle SVC
svc_model = SVC(gamma='auto')

# GridSearchCV avec validation croisée
grid_search_svc = GridSearchCV(svc_model, param_grid_svc, cv=5, scoring='accuracy', n_jobs=-1)

# Entraînement du modèle
grid_search_svc.fit(X_train, y_train)

# Prédiction sur les données de test
predictions_svm = grid_search_svc.predict(X_test)

# Affichage du rapport de classification
print(classification_report(y_test, predictions_svm))


In [ ]:
import joblib

# Sauvegarder le modèle GridSearchCV entraîné
joblib.dump(grid_search_svc, 'grid_search_svc_model.pkl')
print("Modèle sauvegardé sous le nom 'grid_search_svc_model.pkl'")

In [3]:
# Charger le modèle enregistré
import joblib
grid_search_svc = joblib.load('grid_search_svc.pkl')
print("Le modèle a été chargé avec succès")

# Faire des prédictions sur X_test
loaded_predictions = grid_search_svc.predict(X_test)

# Afficher le rapport de classification
print(classification_report(y_test, loaded_predictions))


FileNotFoundError: [Errno 2] No such file or directory: 'grid_search_svc.pkl'

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

plt.figure()
plt.plot(loss, 'b' , label = 'Training loss')
plt.plot(val_loss, 'm' , label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
plt.figure()
plt.plot(acc, 'b' , label = 'Training acc')
plt.plot(val_acc, 'm' , label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()

# 5 - Processus sur une image

In [ ]:
def process_frame(image, mode="fast"):

    # face detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    if mode == "fast":
        matches = hog_detector(gray,1)
    else:
        matches = cnn_detector(gray,1)
        matches = [m.rect for m in matches]

    for rect in matches:

        # face landmarks
        landmarks = pose68(gray, rect)

        # face encoding
        encoding = cnn_encoder.compute_face_descriptor(image, landmarks)

        # face classification
        label = "label"

        # draw box
        cv2.rectangle(image, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
        y = rect.top() - 15 if rect.top() - 15 > 15 else rect.bottom() + 25
        cv2.putText(image, label, (rect.left(), y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    return image

# 6 - Processus sur une vidéo

In [ ]:
import cv2
import numpy as np

def process_movie(video_path, model, face_cascade, emotion_to_emoji):
    """
    Traite une vidéo pour prédire les émotions sur chaque frame et les afficher.
    """
    # Charger la vidéo
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Détection des visages
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Prédiction d'émotions et affichage
        for (x, y, w, h) in faces:
            face = frame[y:y + h, x:x + w]
            resized_face = cv2.resize(face, (128, 128))  # Redimensionner pour le modèle
            face_features = extract_face_features(resized_face)  # Fonction d'extraction des caractéristiques
            emotion = model.predict([face_features])

            # Convertir l'émotion en avatar
            emoji_image = emotion_to_avatar(emotion[0], emotion_to_emoji)
            if emoji_image is not None:
                # Positionner l'emoji sur le visage détecté
                emoji_resized = cv2.resize(emoji_image, (50, 50))  # Ajuster la taille de l'emoji
                frame[y:y + emoji_resized.shape[0], x + w + 10:x + w + 60] = emoji_resized

            # Afficher le cadre du visage et l'émotion
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(frame, emotion[0], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        # Afficher la vidéo avec les prédictions d'émotions
        cv2.imshow("Emotion Prediction in Movie", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# 7- Emotions to avatar

In [10]:
import numpy as np
# Recharger les classes sauvegardées
label_classes = np.load('label_classes.npy')

# Initialiser un nouveau label encoder avec les classes
label_encoder = LabelEncoder()
label_encoder.classes_ = label_classes

In [12]:
import os

def load_emotion_avatars(emotions_path):
    """
    Charge les images des avatars correspondant aux émotions.
    
    Parameters:
        emotions_path (str): Chemin vers le dossier contenant les images des avatars.
    
    Returns:
        dict: Dictionnaire associant chaque émotion à son image.
    """
    emotion_files = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
    emotion_to_emoji = {}
    for emotion in emotion_files:
        image_path = os.path.join(emotions_path, f"{emotion}.png")
        if os.path.exists(image_path):
            emotion_to_emoji[emotion] = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)  # Lire avec la transparence
        else:
            print(f"Image manquante pour l'émotion: {emotion}")
    return emotion_to_emoji

In [14]:
def emotion_to_avatar(emotion, emotion_to_emoji):
    """
    Récupère l'image de l'avatar correspondant à une émotion donnée.
    
    Parameters:
        emotion (str): Le label de l'émotion détectée.
        emotion_to_emoji (dict): Dictionnaire associant chaque émotion à son image.
    
    Returns:
        numpy.ndarray: Image de l'avatar correspondant à l'émotion ou None si non trouvé.
    """
    return emotion_to_emoji.get(emotion, None)

In [16]:
def extract_face_features(face, cnn_encoder, face_landmarks_func):
    """
    Extrait les descripteurs faciaux à l'aide du modèle CNN de Dlib.
    """
    # Extraire les repères faciaux à partir de l'image du visage
    landmarks = face_landmarks_func(face)

    # Calculer les descripteurs faciaux
    features = np.array(cnn_encoder.compute_face_descriptor(face, landmarks))
    return features

In [32]:
import cv2
import numpy as np
from fer import FER
import joblib

def live_webcam_interpret(model, emotion_to_emoji, label_encoder=None):
    """
    Capture la vidéo en direct depuis la webcam et prédit les émotions en temps réel.
    
    Args:
        model: Modèle KNN entraîné sur les vecteurs émotionnels (7 dimensions).
        emotion_to_emoji: Dictionnaire mappant les émotions aux images d'emojis.
        label_encoder: LabelEncoder utilisé pour encoder/décoder les étiquettes d'émotion.
    """
    cap = cv2.VideoCapture(0)  # Activer la webcam par défaut
    detector = FER(mtcnn=True)  # Initialiser FER pour la détection d'émotions
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Échec de la capture de la vidéo")
            break
        
        # Convertir le cadre en RGB (car FER utilise RGB)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Détection des émotions
        emotion_data = detector.detect_emotions(rgb_frame)
        
        if emotion_data:  # Vérifier si des émotions sont détectées
            emotions = emotion_data[0]['emotions']  # Prendre le premier visage détecté
            emotion_vector = list(emotions.values())  # Extraire le vecteur d'émotions (7 valeurs)
            
            # Prédiction avec le modèle KNN
            emotion_index = model.predict([emotion_vector])[0]
            emotion_label = label_encoder.inverse_transform([emotion_index])[0] if label_encoder else emotion_index
            
            # Charger et afficher l'emoji correspondant
            emoji_image = emotion_to_emoji.get(emotion_label, None)
            # Vérifier si l'emoji a un canal alpha (transparence)
            if emoji_image is not None:
                # Redimensionner l'emoji à une taille plus petite
                emoji_resized = cv2.resize(emoji_image, (50, 50))
            
                
                if emoji_resized.shape[2] == 4:
                    # Convertir RGBA en BGR (en ignorant le canal alpha)
                    emoji_resized = cv2.cvtColor(emoji_resized, cv2.COLOR_RGBA2BGR)
                    emoji_resized = cv2.cvtColor(emoji_resized, cv2.COLOR_RGB2BGR)
            
                # Ajouter l'emoji sur l'image
                x_offset, y_offset = 10, 10  # Coordonnées pour placer l'emoji
                frame[y_offset:y_offset + emoji_resized.shape[0], x_offset:x_offset + emoji_resized.shape[1]] = emoji_resized
        
            # Ajouter l'emoji sur l'image
            x_offset, y_offset = 10, 10  # Coordonnées pour placer l'emoji
            frame[y_offset:y_offset + emoji_resized.shape[0], x_offset:x_offset + emoji_resized.shape[1]] = emoji_resized
            
            # Ajouter un texte d'émotion au-dessus de l'emoji
            cv2.putText(frame, emotion_label, (x_offset, y_offset + 70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        frame_height, frame_width = frame.shape[:2]
        text = "Press 'q' to Quit"
        cv2.putText(frame, text, (x_offset, frame_height - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Afficher la vidéo en direct
        cv2.imshow("Emotion Prediction with Webcam", frame)
        
        # Arrêter la boucle si 'q' est pressé
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Libérer la capture et fermer les fenêtres
    cap.release()
    cv2.destroyAllWindows()


In [34]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

grid_search_knn = joblib.load('grid_search_knn.pkl')
label_classes = np.load('label_classes.npy')  # Charger les classes
label_encoder = LabelEncoder()
label_encoder.classes_ = label_classes

# Charger les avatars des émotions
emotions_path = "emotions"  # Dossier contenant les images PNG des avatars
emotion_to_emoji = load_emotion_avatars(emotions_path)

# Tester la fonction
live_webcam_interpret(grid_search_knn, emotion_to_emoji, label_encoder)